# Item 26: Define Function Decorators with `functools.wraps`

In [1]:
# A decorator has the ability to run additional code before and after each call to a function it wraps.
# For example, say that we want to print the arguments and return the value of a function call
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__} ({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

In [5]:
# We can apply this decorator to a function using the @ symbol
@trace
def fibonacci(n):
    """Returns the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n -1))

Using the `@` symbol is equivalent to calling the decorator on the function it wraps and assigning the return value to the original name in the same scope:

`fibonacci = trace(fibonacci)`

In [6]:
# The decorated function runs the wrapper code before and after fibonacci runs
fibonacci(4)

fibonacci ((0,), {}) -> 0
fibonacci ((1,), {}) -> 1
fibonacci ((2,), {}) -> 1
fibonacci ((1,), {}) -> 1
fibonacci ((0,), {}) -> 0
fibonacci ((1,), {}) -> 1
fibonacci ((2,), {}) -> 1
fibonacci ((3,), {}) -> 2
fibonacci ((4,), {}) -> 3


3

In [7]:
# This works well, but it has an unintended side effect. The value returned by the decorator doesn't think it's
# named fibonacci
print(fibonacci)

<function trace.<locals>.wrapper at 0x7fc5185831f0>


The reason this happens is because the `trace` function returns the wrapper defined within its body. The `wrapper` functions is what's assigned to the fibonacci name in the containing module. This behavior is problematic because it undermines tools that do introspection. 

In [8]:
# For example, the help() built-in function is useless when called in the decorated fibonacci function. It should
# instead print out the docstring defined above ('Returns the n-th Fibonacci number')
help(fibonacci)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In [9]:
# Objects serializers break because they cant determnine the location of the original function that was decorated
import pickle

pickle.dumps(fibonacci)

AttributeError: Can't pickle local object 'trace.<locals>.wrapper'

The solution to this is to use the `wraps` helper function from the `functools`  built-in module. This is a decorator that helps you write decorators. When we apply it to the `wrapper` function, it copies all of the important metadata abut the inner function to the outer function.

In [24]:
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__} ({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

@trace
def fibonacci(n):
    """Returns the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n -1))

In [25]:
help(fibonacci)

Help on function fibonacci in module __main__:

fibonacci(n)
    Returns the n-th Fibonacci number



In [26]:
# The pickle object serializer also works
print(pickle.dumps(fibonacci))

b'\x80\x04\x95\x1a\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\tfibonacci\x94\x93\x94.'
